# Scraping Toronto Postal Codes from Wikipedia
-- Pt3 - Explore and cluster the neighborhoods in Toronto.

Skip down to section <b>PART 3 -- Explore and cluster the neighborhoods in Toronto.</b><br /> 
This is part 2 of the assignment. Take Part 1 postal Code list and use it to pull Lat/Long for each code. 

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#import requests # library to handle requests
#import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation



In [3]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

print('geopy')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [4]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
print('ipython')

ipython


In [5]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print('pandas')

pandas


In [6]:
!python3 -m pip install folium

     |████████████████████████████████| 102kB 9.9MB/s eta 0:00:01


In [7]:
import folium

In [8]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [9]:
# verify packages are installed -- folium?
!pip list --isolated

Package                            Version  
---------------------------------- ---------
absl-py                            0.7.0    
alabaster                          0.7.12   
anaconda-client                    1.7.2    
anaconda-project                   0.8.2    
arcgis                             1.6.0    
asn1crypto                         0.24.0   
astor                              0.7.1    
astroid                            2.1.0    
astropy                            3.1.1    
astunparse                         1.6.2    
atomicwrites                       1.3.0    
attrs                              18.2.0   
autoai-libs                        1.10.5   
Babel                              2.6.0    
backcall                           0.1.0    
backports.os                       0.1.1    
backports.shutil-get-terminal-size 1.0.0    
beautifulsoup4                     4.7.1    
biopython                          1.72     
bitarray                           0.8.3    
bkcharts  

In [10]:
# download wikipedia 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
webcopy = requests.get(url).text
postaldata = BeautifulSoup(webcopy, 'lxml')

In [11]:
# create the new Toronto Dataframe
columnlabel = ['Postalcode','Borough','Neighborhood']
tor_codes = pd.DataFrame(columns = columnlabel)

In [12]:
tor_codes.head()

,Postalcode,Borough,Neighborhood


In [13]:
# read through table to get postcode, borough, neighborhood 
wikicontent = postaldata.find('div', class_='mw-parser-output')
table = wikicontent.table.tbody
postcode = 0
borough = 0
neighborhood = 0

In [14]:
tor_codes.head()

,Postalcode,Borough,Neighborhood


In [15]:
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n').replace(']','')  # postcode = td.text
            i = i + 1
            # print(postcode,"tim1",i)  #added
        elif i == 1:
            borough = td.text.strip('\n').replace(']','')   # borough1 = td.text
            i = i + 1
            # print(borough,i)  #added
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
            # print(neighborhood,"tim3",i)  #added
    tor_codes = tor_codes.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [16]:
tor_codes.head()

,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [17]:
# clean dataframe 
tor_codes = tor_codes[tor_codes.Borough!='Not assigned']
tor_codes = tor_codes[tor_codes.Borough!= 0]
tor_codes.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,tor_codes.shape[0]):
    if tor_codes.iloc[i][2] == 'Not assigned':
        tor_codes.iloc[i][2] = tor_codes.iloc[i][1]
        i = i+1

In [18]:
tor_codes.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
df = tor_codes.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
# drop burroughs with none assigned
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [20]:

tor_codes.head(20)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [21]:
print(tor_codes.shape)

(103, 3)


### PART 2 -- Get Long/Lat

In [22]:
# Read the cvs file and convert it to a dataframe

url='http://cocl.us/Geospatial_data'
df_pcodesll=pd.read_csv(url)
df_pcodesll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
#Change "Postal Code" to "Postalcode" for proper merger
df_pcodesll.columns = ['Postalcode', 'Latitude', 'Longitude']
df_pcodesll.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [187]:
nhs=pd.merge(tor_codes,df_pcodesll, how='right', on = 'Postalcode')
nhs.head(111)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Part 3 - Explore and cluster the neighborhoods in Toronto. 


Part A) I worked with the Borough that contained the word Toronto and then used CircleMap flags to identify each on a map in different colors.

Part B) I accessed FourSquare to find coffee places near the Neighborhood that has the University of Toronto so I can meet a friend for coffee.

Hints from Course Admin
- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visalize your neighborhoods and how they cluster together.

### Pt A -- Group neighborhoods with like burroughs

In [25]:

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))

Full Toronto -- Postal Code DF

In [26]:
grp = nhs.groupby([ 'Borough','Postalcode', 'Latitude','Longitude'])
tor_neigh = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
tor_neigh.head(100)

,Borough,Postalcode,Latitude,Longitude,Neighborhood
0,Central Toronto,M4N,43.728020,-79.388790,Lawrence Park
1,Central Toronto,M4P,43.712751,-79.390197,Davisville North
2,Central Toronto,M4R,43.715383,-79.405678,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,43.704324,-79.388790,Davisville
4,Central Toronto,M4T,43.689574,-79.383160,"Moore Park, Summerhill East"
5,Central Toronto,M4V,43.686412,-79.400049,"Summerhill West, Rathnelly, South Hill, Forest..."
6,Central Toronto,M5N,43.711695,-79.416936,Roselawn
7,Central Toronto,M5P,43.696948,-79.411307,"Forest Hill North & West, Forest Hill Road Park"
8,Central Toronto,M5R,43.672710,-79.405678,"The Annex, North Midtown, Yorkville"
9,Downtown Toronto,M4W,43.679563,-79.377529,Rosedale


Get the Lat/Long of Toronto, ON

In [27]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


<B>Filter the DF to only work with "Toronto area" Boroughs. </b>

In [149]:
c_toronto_data = tor_neigh[tor_neigh['Borough'].str.contains('Toronto', na = True)] # 
c_toronto_data.head(44)

,Borough,Postalcode,Latitude,Longitude,Neighborhood
0,Central Toronto,M4N,43.728020,-79.388790,Lawrence Park
1,Central Toronto,M4P,43.712751,-79.390197,Davisville North
2,Central Toronto,M4R,43.715383,-79.405678,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,43.704324,-79.388790,Davisville
4,Central Toronto,M4T,43.689574,-79.383160,"Moore Park, Summerhill East"
5,Central Toronto,M4V,43.686412,-79.400049,"Summerhill West, Rathnelly, South Hill, Forest..."
6,Central Toronto,M5N,43.711695,-79.416936,Roselawn
7,Central Toronto,M5P,43.696948,-79.411307,"Forest Hill North & West, Forest Hill Road Park"
8,Central Toronto,M5R,43.672710,-79.405678,"The Annex, North Midtown, Yorkville"
9,Downtown Toronto,M4W,43.679563,-79.377529,Rosedale


In [148]:
print('The "Contains Toronto" dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

The "Contains Toronto" dataframe has 4 boroughs and 39 neighborhoods.


In [154]:
c_toronto_data = tor_neigh[tor_neigh['Borough'] == 'Central Toronto'].reset_index(drop=True)
#c_toronto_data = tor_neigh[tor_neigh['Borough'].str.contains('Toronto', na = True)] # 
c_toronto_data.head(55)

,Borough,Postalcode,Latitude,Longitude,Neighborhood
0,Central Toronto,M4N,43.728020,-79.388790,Lawrence Park
1,Central Toronto,M4P,43.712751,-79.390197,Davisville North
2,Central Toronto,M4R,43.715383,-79.405678,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,43.704324,-79.388790,Davisville
4,Central Toronto,M4T,43.689574,-79.383160,"Moore Park, Summerhill East"
5,Central Toronto,M4V,43.686412,-79.400049,"Summerhill West, Rathnelly, South Hill, Forest..."
6,Central Toronto,M5N,43.711695,-79.416936,Roselawn
7,Central Toronto,M5P,43.696948,-79.411307,"Forest Hill North & West, Forest Hill Road Park"
8,Central Toronto,M5R,43.672710,-79.405678,"The Annex, North Midtown, Yorkville"


In [155]:
dt_toronto_data = tor_neigh[tor_neigh['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
#dt_toronto_data = tor_neigh[tor_neigh['Borough'].str.contains('Toronto', na = True)] # 
dt_toronto_data.head(55)

,Borough,Postalcode,Latitude,Longitude,Neighborhood
0,Downtown Toronto,M4W,43.679563,-79.377529,Rosedale
1,Downtown Toronto,M4X,43.667967,-79.367675,"St. James Town, Cabbagetown"
2,Downtown Toronto,M4Y,43.665860,-79.383160,Church and Wellesley
3,Downtown Toronto,M5A,43.654260,-79.360636,"Regent Park, Harbourfront"
4,Downtown Toronto,M5B,43.657162,-79.378937,"Garden District, Ryerson"
5,Downtown Toronto,M5C,43.651494,-79.375418,St. James Town
6,Downtown Toronto,M5E,43.644771,-79.373306,Berczy Park
7,Downtown Toronto,M5G,43.657952,-79.387383,Central Bay Street
8,Downtown Toronto,M5H,43.650571,-79.384568,"Richmond, Adelaide, King"
9,Downtown Toronto,M5J,43.640816,-79.381752,"Harbourfront East, Union Station, Toronto Islands"


In [156]:
e_toronto_data = tor_neigh[tor_neigh['Borough'] == 'East Toronto'].reset_index(drop=True)
e_toronto_data.head(55)

,Borough,Postalcode,Latitude,Longitude,Neighborhood
0,East Toronto,M4E,43.676357,-79.293031,The Beaches
1,East Toronto,M4K,43.679557,-79.352188,"The Danforth West, Riverdale"
2,East Toronto,M4L,43.668999,-79.315572,"India Bazaar, The Beaches West"
3,East Toronto,M4M,43.659526,-79.340923,Studio District
4,East Toronto,M7Y,43.662744,-79.321558,"Business reply mail Processing Centre, South C..."


In [157]:
w_toronto_data = tor_neigh[tor_neigh['Borough'] == 'West Toronto'].reset_index(drop=True)
w_toronto_data.head(55)

,Borough,Postalcode,Latitude,Longitude,Neighborhood
0,West Toronto,M6H,43.669005,-79.442259,"Dufferin, Dovercourt Village"
1,West Toronto,M6J,43.647927,-79.419750,"Little Portugal, Trinity"
2,West Toronto,M6K,43.636847,-79.428191,"Brockton, Parkdale Village, Exhibition Place"
3,West Toronto,M6P,43.661608,-79.464763,"High Park, The Junction South"
4,West Toronto,M6R,43.648960,-79.456325,"Parkdale, Roncesvalles"
5,West Toronto,M6S,43.651571,-79.484450,"Runnymede, Swansea"


In [171]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map

for lat, lng, borough, neighborhood in zip(dt_toronto_data['Latitude'], dt_toronto_data['Longitude'], dt_toronto_data['Borough'], dt_toronto_data['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, borough, neighborhood in zip(c_toronto_data['Latitude'], c_toronto_data['Longitude'], c_toronto_data['Borough'], c_toronto_data['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, borough, neighborhood in zip(e_toronto_data['Latitude'], e_toronto_data['Longitude'], e_toronto_data['Borough'], e_toronto_data['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, borough, neighborhood in zip(c_toronto_data['Latitude'], w_toronto_data['Longitude'], w_toronto_data['Borough'], w_toronto_data['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='green',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

### 3b - Find the Neighborhood with the University of Toronto on the map - Then find nearby coffee shops

The goal is to Create a map of the nearest coffee shops near the University or Toronto so I can meet up with a friend.


In [189]:
target_loc = tor_neigh[tor_neigh['Neighborhood'].str.contains('University', na = True)] # 
target_loc 

,Borough,Postalcode,Latitude,Longitude,Neighborhood
21,Downtown Toronto,M5S,43.662696,-79.400049,"University of Toronto, Harbord"
63,North York,M3J,43.767980,-79.487262,"Northwood Park, York University"


The Univversity of Toronto is in Loc=21, so we'll start there to find where the coffee shops are.

In [190]:

neighborhood_latitude = tor_neigh.loc[21, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tor_neigh.loc[21, 'Longitude'] # neighborhood longitude value

neighborhood_name = tor_neigh.loc[21, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of University of Toronto, Harbord are 43.6626956, -79.4000493.


### Find places to eat nearest the University of Toronto for using FourSquare

In [191]:
CLIENT_ID = 'xxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'yyyyyyyyyyy' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KHKXZNF5X32XKEAPPSZZNQ3EGJU2FGAIKE00LUQBMKNP4HGG
CLIENT_SECRET:IXUDUIXQH1QHNPWTPL4YDCVEWNJVBGJ1DD0GGFRBHMBNUJDA


Search a 800 meter (about 1/2 mile) area around the Airport for "restaurants".

In [192]:
latitude = neighborhood_latitude
longitude = neighborhood_longitude

search_query = 'restaurants'
radius = 800
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=KHKXZNF5X32XKEAPPSZZNQ3EGJU2FGAIKE00LUQBMKNP4HGG&client_secret=IXUDUIXQH1QHNPWTPL4YDCVEWNJVBGJ1DD0GGFRBHMBNUJDA&ll=43.6626956,-79.4000493&v=20180605&radius=800&limit=100'

In [193]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0b71a86da43a2157533e21'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'University of Toronto',
  'headerFullLocation': 'University of Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 131,
  'suggestedBounds': {'ne': {'lat': 43.669895607200004,
    'lng': -79.3901151046533},
   'sw': {'lat': 43.655495592799994, 'lng': -79.40998349534671}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5362c366498e602fbe1db395',
       'name': 'Yasu',
       'location': {'address': '81 Harbord St.',
        'lat': 43.66283719650635,
        'lng': -79.40321739973975,
        'labeledLatLngs': [{'label': 'display',
        

In [194]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



My end result was that I found 13 businesses within the 800 meters of the Airport.

In [195]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(125)

,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Rasa,Restaurant,43.662757,-79.403988
2,Her Father's Cider Bar + Kitchen,Beer Bar,43.662448,-79.404703
3,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746
4,Almond Butterfly,Bakery,43.662836,-79.403365
5,Piano Piano,Italian Restaurant,43.662949,-79.402898
6,Harbord House,Bar,43.662466,-79.405410
7,Akai Sushi,Sushi Restaurant,43.662470,-79.404946
8,Fresh on Bloor,Vegetarian / Vegan Restaurant,43.666755,-79.403491
9,Sivananda Yoga Centre,Yoga Studio,43.662754,-79.402951


I had too many restaurants so I filtered to just include: Coffee, Cafe, and Donut.

In [213]:

pub_places = nearby_venues[nearby_venues['categories'].str.contains('Coffee|Cafe|Donut', na = True)] # 
pub_places.head()

,name,categories,lat,lng
25,A & C World,Gaming Cafe,43.657409,-79.399847
75,Starbucks,Coffee Shop,43.668615,-79.397387
77,Krispy Kreme Doughnuts,Donut Shop,43.655834,-79.399417
81,Hopper Coffee,Coffee Shop,43.657479,-79.402691
86,B Espresso Bar,Coffee Shop,43.668143,-79.396087


In [214]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
print('{} coffee places were near to the University.'.format(pub_places.shape[0]))

100 venues were returned by Foursquare.
6 coffee places were near to the University.


In [218]:
latitude = neighborhood_latitude
longitude = neighborhood_longitude

# create map of Toronto using latitude and longitude values
map_tor_pubs = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, categories, name in zip(pub_places['lat'], pub_places['lng'], pub_places['name'], pub_places['categories']):
    label = '{}, {}'.format(categories,name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.9,
        parse_html=False).add_to(map_tor_pubs)  
    
map_tor_pubs